# Settings

In [1]:
import sys, os, platform, psycopg2, gc, joblib, warnings, json, pickle, datetime, glob, time, uuid
from tqdm import tqdm
from datetime import datetime
import numpy as np
import pandas as pd, numpy as np
from __future__ import annotations
from typing import Any, Iterable, List, Dict, Optional, TypedDict
from IPython.display import Markdown
from mcbs.config import settings
from mcbs.llm_api.hm_langchain_generator import LlmApiGenerator
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(message)s')

In [2]:
main_path = 'prompting'
user_scenario_path = main_path + '/user_scenario'

In [3]:
examples = []
example_fnames = glob.glob(f'{user_scenario_path}/*.txt')
for example_fname in example_fnames:
    with open(example_fname, "r", encoding="utf-8") as f:
        example_text = f.read()
    examples.append(example_text)
print("#Examples", len(examples))
examples

#Examples 15


['\n# [1] User Information\n- **User ID** : 44393\n- **Session ID** : 24\n- **Email** : qoorooqooroo@naver.com\n- **Username** : 최욱진\n- **Nickname** : 땡자오빠야\n- **Phone** : 01075795020\n- **Feedback T/F?** : False\n\n\n\n# [2] Pre-Questionnaire\n### Dog Care Counseling Pre-Survey Details\n#### 0. Guardian Information\n    - Age : 37\n    - Gender : 남\n\n#### 1. Dog Information\n    - Name : 땡자\n    - Age : 5.5\n    - Weeks since adoption : 39 weeks old\n    - sex : 암컷 (neutered \'예\')\n    - breed : 푸들\n    - Relationship with user : 보조양육자(가족 중 주양육자가 따로 있으며, 먹이 급여, 놀이 등 생활 일부만 수행)\n    - Adoption method : 펫샵에서 입양\n    - Walking schedule : 10 times/week, 2 times/day, 20 minutes each\n#### 2. Living Environment\n    - Type of residence : 주거용 공간(아파트, 원룸, 단독주택 등)\n        • Number of rooms : 4\n        • Kept tethered outdoors? : 아니오\n    - Family composition :\n        • Number of human family members : 2\n        • Number of dogs : 1\n        • Number of non-dog pets : 0\n    - Financial 

# Main

In [4]:
def generate_task_contents(query, params, tagname):
    llm_api_generator=LlmApiGenerator()
    response = llm_api_generator.generate_message(
        query=query,
        params=params,
        message_history=None,
        tagname=tagname
    )
    return response

## Generate pre-questionnaire response

In [5]:
# 사전 설문 응답 기록(fake)
preq_contexts_output_path = f'{user_scenario_path}/preq_contexts.pkl'
if len(glob.glob(preq_contexts_output_path)) > 0 :
    preq_contexts = joblib.load(preq_contexts_output_path)
    to_extract_preq = False
else :
    to_extract_preq = True

In [6]:
system_instruction = """
### Role
- 인풋으로 유저가 반려견 양육 상담 채팅 서비스를 이용한 기록이 제공될거야.
- 이 내용을 토대로, 이 유저가 반려견 사전설문에 어떻게 응답을 하였을지, 역으로 설문 응답 기록을 추출해줘.
- response_schema를 통해 설문 응답 기록을 제시해줄테니, 꼭 준수해.

### response_schema: 
- Just return the JSON object. Only return '{key: value, key: value, ...}` format. ```json와 같이 불필요한 텍스트는 절대 출력하지 마세요.
"""
response_schema = {
    "type": "object",
    "properties": {
        "animal_type": {
            "type": "string",
            "enum": ["Dog"]
        },
        "pet_name": {
            'type': 'string',
            'description': '반려견의 이름'
        },
        "pet_birth_date": {
            'type': 'string',
            'description': '반려견의 생년월일(YYYY-MM-DD) e.g. 2022-02-01'
        },
        "pet_adopted_at": {
            'type': 'string',
            "description": "반려견의 입양일(YYYY-MM-DD) e.g. 2022-05-03"
        },
        "pet_gender": {
            "type": "string",
            "enum": ["M", "F"]
        },
        "pet_is_neutered": {
            "type": "boolean",
            "description": "반려견의 중성화 여부"
        },
        "pet_breed": {
            "type": "string",
            "description": "반려견의 품종"
        },
        "env1": {
            "type": "string",
            "description": "Q.보호자님은 반려동물의 양육에서 어떤 역할을 맡고 계신가요?",
            "enum": ["주양육자(먹이 급여, 산책, 놀이, 기타 생활 대부분을 책임짐)", 
                     "보조양육자(가족 중 주양육자가 따로 있으며, 먹이 급여, 놀이 등 생활 일부만 수행)",
                     "반려동물과 함께 거주하지 않음"]
        },
        "env2": {
            "type": "string",
            "description": "Q.반려동물과 어떻게 만나게 되셨나요? <선택지> ['키우던 반려동물이 출산'|'펫샵에서 입양'|'브리더로부터 입양가정 분양'|'동물보호센터로부터 입양'|'특수목적견 입양'|'길에서 구조'|'기타(직접 입력)']  (기타면 추가 내용을 출력하세요.)",
        },
        "env3": {
            "type": "string",
            "description": "Q.반려동물이 어떤 공간에서 살고 있나요? <선택지> ['실외(마당, 건물 앞 등)'|'업무 공간(회사 사무실, 카페 등)'|'주거용 공간(아파트, 원룸, 단독주택 등)'|'기타(직접 입력)'] (기타면 추가 내용을 출력하세요.)",
        },
        "env4": {
            "type": "number",
            "description": "Q.반려동물이 거주하는 공간에는 방이 몇 개 있나요?(숫자만 입력하세요)"
        },
        "env5": {
            "type": "string",
            'enum': ['예', '아니오'],
            "description": "Q.반려동물을 평소에 묶어두고 키우나요?"
        },
        "env6": {
            "type": "string",
            "description": "Q.지난 1년 동안 월평균 반려동물 양육 비용은 어느 정도였나요? (만원) // (모든 반려동물에 대한 양육 비용을, 숫자만 입력해 주세요.)"
        },
        "env7": {
            "type": "number",
            "description": "Q.가족 구성원 중 '사람' 가족 수는 모두 몇 명인가요? (숫자만 입력해 주세요.)"
        },
        "env8": {
            "type": "string",
            "enum": ['3세 미만 영유아', '3세~6세 미만 유아', '6세~10세 미만 아동', '65세 이상 노인', '없음'],
            "description": "Q.반려동물과 함께 거주하는 구성원 중 해당하는 인원이 있다면 모두 선택해 주세요."
        },
        "env9": {
            "type": "number",
            "description": "현재 함께 거주하는 반려동물은 모두 몇 마리인가요? (선택한 반려동물 포함)"
        },
        "life_walk_day": {
            "type": "number",
            "description": "Q.(최근 3개월간 산책 주기) 하루 평균 몇 회 했나요?"
        },
        "life_walk_hour": {
            "type": "number",
            "description": "Q.(최근 3개월간 산책 주기) 한 번 할 때 몇 분 했나요?"
        },        
        "life_walk_week": {
            "type": "number",
            "description": "Q.(최근 3개월간 산책 주기) 일주일에 몇 회꼴로 했나요?"
        },        
        "life_play": {
            "type": "string",
            "description": "Q.(단순 걷기 산책 외) 반려동물과 어떤 놀이들을 하는지, 횟수와 시간도 함께 알려 주세요."
        },        
        "life_human": {
            "type": "number",
            "description": "Q.하루에 사람 가족과 함께 보내는 시간을 알려주세요.	(지난 3개월간을 기준으로, 스마트폰, pc 사용 없이 함께한 평균 시간을 입력해 주세요.)"
        },        
        "life_alone": {
            "type": "number",
            "description": "Q.(지난 3개월간 기준) 평소 반려동물이 사람 없이 집에 있는 시간은 하루 평균 몇 시간인가요?"
        },      
        "life_space": {
            "type": "array",
            "items": {
                "type": "string",
                "enum": ["문이 있는 플라스틱 크레이트(바리, 행복 켄넬 등)", "텐트형 하우스 (문이 없는)", "방석이나 매트", "몇 군데 본인이 좋아하는 자리", "개가 쉴 수 있는 특정 자리는 없다"],
                "description": "Q.반려동물이 휴식할 수 있는 공간이 따로 있나요? 있다면 가장 가까운 형태를 선택해 주세요."
            }
        },      
        "pref_rewards": {
            "type": "string",
            "description": "Q.트릿(간식) 보상물 이외에 반려동물에게 보상물로 사용할 수 있는 것이 있으면 적어 주세요. 예: 장난감 중 공을 제일 좋아해요, 터그토이를 제일 좋아해요 (단단한거 보다는 부드러운 것), 스킨십을 제일 좋아해요 등"
        },           
        "med_disease": {
            "type": "string",
            "description": "Q.지난 6개월간 병원 진료를 받거나 입원 또는 수술을 받은 적이 있다면, 어떠한 질환 때문인지 알려주세요. (처방받은 약물에 대해서도 기술하시면 더욱 좋습니다.)"
        },           
        "med_allergy": {
            "type": "string",
            "description": "Q.반려동물이 알러지를 가지고 있다면 알려 주세요. 없으면 '없다'라고 적어 주세요."
        },           
        "social_interactions": {
            "type": "string",
            "enum": ['5명 미만', '5~10명', '11~20명', '21명~30명', '31명~40명', '40명 초과', '잘 모르겠다.'],
            "description": "Q.반려동물이 사회화 시기에 몇 명의 사람들(가족, 친적, 친구, 지인, 미용사, 수의사, 수의테크니션 등)과 접촉하고 상호작용을 하였나요? 	※ 사회화 시기 : 생후 3~16주"
        },           
        "social_trains": {
            "type": "array",
            "items": {
                "type": "string",
                "enum": ["손", "앉아", "기다려", "멈춰", "엎드려", "돌아", "콜백(부르면 오기)", "하우스(개집과 같은 지정된 공간으로 돌아가기)", "터치(사람의 손에 코나 발을 가져다 대기)", "브이(사람의 손가락 브이 사이에 턱을 대는 행동)", "매트", "줄 당기지 않고 산책하기(산책매너)", "없음", "기타"],
                "description": "Q.다음 중 낯선 자극이 없는 환경에서 반려동물이 현재 수행할 수 있는 '신호(큐)'를 모두 골라 주세요.(지시했을 때 70% 이상의 빈도로 수행하는 것만 선택해 주세요.) (복수선택 가능)"
            }
        },           
        "fear_triggers": {
            "type": "string",
            "description": "Q.반려동물이 유난히 경계하거나 불편해 하는 대상이나 상황이 있다면, 어떤 자극에 어떻게 반응하는지 구체적으로 적어 주세요. 예: 특정 사람, 소리, 물건, 공간 등. 예를 들어 초인종 소리가 들릴 때 짖는다거나, 특정 스타일의 남성이 다가올 때 몸을 숨긴다거나 짖는 행동을 포함할 수 있어요."
        },           
        "fear_touch": {
            "type": "array",
            "items": {
                "type": "string",
                "description": "['전반적으로 몸 전체'|'얼굴'|'발'|'엉덩이 및 생식기'|'안아서 들기'|'뽀뽀하기'|'없음'|'기타(직접 입력)']  (기타면 추가 내용을 출력하세요.)"
            },
            "description": "Q.반려동물이 보호자님께 '거부 반응'을 보이는 신체 부위/스킨십 방식을 알려 주세요. 거부 반응 예시: 고개를 반대로 돌리거나 몸을 피하기, 얼어 붙기, 리킹(뭔가를 먹거나 마시지 않았는데 혀로 입이나 코를 핥는 행동), 으르렁거리기, 입질 등"
        },           
        "fear_stranger": {
            "type": "string",
            "enum": ['별다른 반응을 보이지 않는다.', '약간 흥분하지만 경계하지 않고 잘 어울린다.', '사람을 너무 좋아하여 과하게 흥분한다.', '짖으면서 도망가거나 소파 같은 가구 아래로 숨지만 물려고 달려들지는 않는다.', '짖으면서 상대방에게 달려 들지만 물지는 않는다.', '짖으면서 상대방에게 달려들고 물려고 시도 한다.', '짖지 않고 조용히 있다가 갑자기 사람에게 달려들어 물려고 시도 한다.', '사람을 문 적이 있다.', '해당 경험 없음'],
            "description": "Q.반려동물은 낯선 사람이 집에 방문하는 것에 어떤 반응을 보이나요?"
        },           
        "fear_clinic": {
            "type": "array",
            "items":{
                "type": "string",
                "description": "['해당 상황이 없었다'|'뻣뻣하게 서서 뚫어져라 본다'|'낮게 으르렁거린다'|'입을 오므리거나 이를 드러내고 위협한다'|'물려고 한다/문다'|'기타(직접 입력)']  (기타면 추가 내용을 출력하세요.)"
            },
            "description": "Q.동물병원에서 진료를 받을 때, 수의사나 간호사 등 접촉하는 사람에게 어떤 반응을 보이나요?(복수응답 가능)"
        },           
        "fear_dog": {
            "type": "string",
            "enum": ['관련 경험 없음', '전혀 그렇지 않다', '약간 그렇지 않다', '보통이다', '약간 그렇다', '매우 그렇다'],
            "description": "Q.반려동물이 다른 반려동물들과 잘 어울려 놀 수 있나요?"
        },           
        "fear_separation": {
            "type": "string",
            "enum": ['관련 경험 없음', '전혀 그렇지 않다', '약간 그렇지 않다', '보통이다', '약간 그렇다', '매우 그렇다'],
            "description": "Q.반려동물이 주 보호자가 가는 곳마다 따라다니려고 하나요?	예시) 4점: 화장실에 가도 따라 들어오려고 한다. / 5점: 외출하려고 하면 집밖까지 쫓아오려고 한다."
        },           
        "behavior_guarding": {
            "type": "string",
            "description": "Q.(지난 3개월간 기준) 일상 속에서 반려동물이 ‘지키려는 듯한 행동’을 보인 적이 있다면 어떤 상황이었는지, 어떤 행동과 어느 정도의 수위로 경계하였는지 알려 주세요. 반려동물은 먹이, 간식, 장난감, 보호자, 특정 물건 등을 지키려고 할 수 있어요. 지키려는 행동은 사람이나 다른 동물의 접근을 경계하여 짖거나 으르렁거리는 일을 말해요."
        },           
        "behavior_separation": {
            "type": "string",
            "description": "Q.보호자가 외출을 준비하거나 외출하여 사람이 없는 집에 남겨졌을 때, 평소와 다르게 보이는 행동이 있다면 알려 주세요.	(배변실수, 문 긁어대기, 물건 파괴, 하울링 등)"
        },           
        "behavior_demand": {
            "type": "array",
            "items": {
                "type": "string",
                "description": "['쳐다보거나 다가와 근처에 있는다'|'앞발로 가볍게 툭툭 건드린다'|'낑낑거리거나 조용히 짖는다'|'흥분한 채로 짖는다'|'보호자의 옷, 악세서리, 신체 일부를 물어서 잡아당긴다'|'큰 소리를 내거나 물건을 해친다'|'보호자를 상처가 나도록 문다'|'기타(직접 입력)']  (기타면 추가 내용을 출력하세요.)"
            },
            "description": "Q.주 보호자가 다른 일에 몰두해 있을 때, 원하는 것이 있으면 어떻게 하나요?(복수선택 가능)"
        },           
        "behavior_impulsive": {
            "type": "string",
            'enum': ['관련 경험 없음', '전혀 그렇지 않다', '약간 그렇지 않다', '보통이다', '약간 그렇다', '매우 그렇다'],
            "description": "Q.호기심이나 즐거움을 주는 자극에 대한 반응이 즉각적인가요? (예: 음식을 보면 달려든다.)"
        },           
        "behavior_overarousal": {
            "type": "string",
            'enum': ['관련 경험 없음', '전혀 그렇지 않다', '약간 그렇지 않다', '보통이다', '약간 그렇다', '매우 그렇다'],
            "description": "Q.흥분했을 때 과도한 신체적 반응을 보이나요?(과도한 신체적 반응: 침 흘리기, 헐떡이기, 소변 보기 등)"
        },           
        "behavior_licking": {
            "type": "string",
            'enum': ['관련 경험 없음', '전혀 그렇지 않다', '약간 그렇지 않다', '보통이다', '약간 그렇다', '매우 그렇다'],
            "description": "Q.피부가 상할 정도로 특정 부위를 계속해서 핥는다."
        },           
        "behavior_denial": {
            "type": "string",
            'enum': ['관련 경험 없음', '전혀 그렇지 않다', '약간 그렇지 않다', '보통이다', '약간 그렇다', '매우 그렇다'],
            "description": "Q.보호자가 반려동물의 요구(예: 원하는 물건이나 행동 요청)에 대해 '안 돼'라고 말하거나, 물러나기, 밀치기, 자리 피하기 등으로 거절했을 때, 반려동물의 요구 행동이 더욱 강해지는 경향이 있나요?"
        },           
        "behavior_aggression": {
            "type": "string",
            "description": "Q.반려동물이 위협적이거나 공격적인 행동을 보인 적이 있나요? 있었다면, 언제 어떤 상황에서, 무엇 때문에, 그리고 얼마나 자주 그런 행동을 했는지 구체적으로 알려주세요. ※ 여기서 말하는 위협/공격 행동에는 흥분 상태에서 짖거나 으르렁대는 행동, 사람이나 다른 동물을 쫓거나 공격하는 행위를 말합니다."
        },           
        "behavior_bite": {
            "type": "array",
            "items": {
                "type": "string",
                "enum": ['없음',
                         '레벨1 - 개가 두려움이나 불안으로 인해 공격적이거나 불쾌한 행동을 보이지만, 이빨이 상대의 피부에 닿지는 않는다.',
                         '레벨2 - 개의 이빨이 상대의 피부에 닿지만, 피부에 구멍이 나지 않는다. 다만 피부에 가벼운 이빨 자국이 생기거나, 이빨이 스치면서 아주 약간의 출혈이 있을 수 있다. (상처의 길이는 2.5cm 이하로, 피부가 뚫리는 깊은 상처는 아님)',
                         '레벨3A - 한 번의 물림으로 인해 피부에 1~4개의 구멍이 생기나, 그 깊이는 개 이빨 길이의 절반보다 얕다. 물린 상대가 손을 급히 빼거나, 개를 떼어내려는 과정 또는 중력(개가 점프하여 물었다 떨어지는 경우)에 의해 상처가 찢어지거나 길게 생길 수 있다.',
                         '레벨3B - 레벨 3A에 해당하는 물림 행동이 여러 번 발생한 경우.',
                         '레벨4 - 한 번의 물림으로 인해 피부에 1~4개의 구멍이 생기며, 그중 최소 한 개의 구멍은 개의 이빨 길이의 절반 이상으로 깊다. 개가 몇 초간 지속적으로 물고 있는 경우 주변에 멍이 들거나, 물고 흔드는 과정에서 양쪽 방향으로 찢어지는 상처가 발생할 수 있다.',
                         '레벨5 - 여러 차례의 공격적 물림 사건이 발생하며, 최소 2회 이상 레벨 4에 해당하는 깊고 심각한 물림이 포함된다. 또한 각각의 공격 사건마다 최소 1회씩 레벨 4의 물림이 나타난다.',
                         '레벨6 - 물림으로 인해 피해자가 사망한 경우.'],
            },
            "description": "Q.반려동물이 다른 사람 혹은 동물을 물었던 적이 있다면, 다음의 묘사 중 어느 정도 수준에 해당하였는지 선택해 주세요.(해당 상황이 여러번 있었을 경우, 복수선택해 주세요.)"
        },           
        "counseling_needs": {
            "type": "string",
            "description": "Q.반려동물의 행동과 관련해, 교육이 필요하다고 생각하는 내용을 모두 적어 주세요. (자세하고 구체적일수록 좋아요.)"
        },
        "dcsi_label": {
            "type": "string",
            "enum": ['눈치빠른 탐험가','느긋한 모범생','침착한 주도자','신비로운 관찰자',
                     '여유로운 전략가','천진난만한 안내자','영리한 부끄럼쟁이','성실한 사색가',
                     '용감한 사냥꾼','진취적인 수호자','재치있는 전략가','선량한 신비주의자',
                     '자유로운 영혼의 협상가','열정적인 탐구자','민첩한 활동가','소심한 흥부자',
                     'NONE'],
            "description": "DCSI 검사 Label. 결과에 대한 언급이 없으면 NONE으로 적어주세요."
        },
        "dcsi_code": {
            "type": "string",
            "enum": ['PICA','PICF','PIWA','PIWF',
                     'PSCA','PSCF','PSWA','PSWF',
                     'EICA','EICF','EIWA','EIWF',
                     'ESCA','ESCF','ESWA','ESWF',
                     'NONE'],
            "description": "DCSI 검사 Code. 결과에 대한 언급이 없으면 None으로 적어주세요."
        }
    },
    "required":['animal_type', 'pet_name', 'pet_birth_date', 'pet_adopted_at', 'pet_gender', 'pet_is_neutered', 'pet_breed', 
                'env1', 'env2', 'env3', 'env4', 'env5', 'env6', 'env7', 'env8', 'env9', 
                'life_walk_day', 'life_walk_hour', 'life_walk_week', 'life_play', 'life_human', 'life_alone', 'life_space', 'pref_rewards', 
                'med_disease', 'med_allergy', 'social_interactions', 'social_trains', 
                'fear_triggers', 'fear_touch', 'fear_stranger', 'fear_clinic', 'fear_dog', 'fear_separation', 
                'behavior_guarding', 'behavior_separation', 'behavior_demand', 'behavior_impulsive', 'behavior_overarousal', 'behavior_licking', 'behavior_denial', 
                'behavior_aggression', 'behavior_bite', 'counseling_needs',
                'dcsi_label', 'dcsi_code']
}
params = {
    "system_instruction": system_instruction + '\n\n' + str(response_schema),
    "model": "gemini-2.5-pro",
    "temperature": 0.2,
}

In [7]:
if to_extract_preq:
    preq_responses = []
    for example in examples :
        preq_response = generate_task_contents(example, params, tagname='preq')
        preq_responses.append(preq_response)

In [8]:
if to_extract_preq:
    preq_contexts = []
    for preq_response in preq_responses :
        preq_response_str = preq_response.replace('```json', '').replace('```', '')
        preq_context = json.loads(preq_response_str)
        preq_contexts.append(preq_context)
    joblib.dump(preq_contexts, preq_contexts_output_path)
print(preq_contexts[-1])

{'animal_type': 'Dog', 'pet_name': '오복', 'pet_birth_date': '2024-01-20', 'pet_adopted_at': '2024-03-16', 'pet_gender': 'F', 'pet_is_neutered': False, 'pet_breed': '진돗개', 'env1': '주양육자(먹이 급여, 산책, 놀이, 기타 생활 대부분을 책임짐)', 'env2': '브리더로부터 입양가정 분양', 'env3': '주거용 공간(아파트, 원룸, 단독주택 등)', 'env4': 2, 'env5': '아니오', 'env6': '5', 'env7': 4, 'env8': '없음', 'env9': 1, 'life_walk_day': 2, 'life_walk_hour': 40, 'life_walk_week': 14, 'life_play': '이갈이 장난감을 가지고 놂', 'life_human': 20, 'life_alone': 2, 'life_space': ['몇 군데 본인이 좋아하는 자리'], 'pref_rewards': '간식', 'med_disease': '없다', 'med_allergy': '없다', 'social_interactions': '5~10명', 'social_trains': ['앉아', '기다려', '엎드려'], 'fear_triggers': '산책 후 발 닦기, 안거나 몸을 만지는 것', 'fear_touch': ['안아서 들기', '발'], 'fear_stranger': '해당 경험 없음', 'fear_clinic': ['기타(뒷걸음질)'], 'fear_dog': '약간 그렇지 않다', 'fear_separation': '전혀 그렇지 않다', 'behavior_guarding': '없다', 'behavior_separation': '없다', 'behavior_demand': ['쳐다보거나 다가와 근처에 있는다'], 'behavior_impulsive': '매우 그렇다', 'behavior_overarousal': '약

## Generate Counseling Guide

In [9]:
# 상담 가이드
counseling_guide_output_path = f'{user_scenario_path}/counseling_guide.pkl'
if len(glob.glob(counseling_guide_output_path)) > 0 :
    counseling_guide = joblib.load(counseling_guide_output_path)
    to_generate_counseling_guide = False
else :
    to_generate_counseling_guide = True

In [10]:
def extract_context_from_preq(info):
    info['current_date'] = datetime.now().strftime('%Y-%m-%d')
    info['pet_age'] = round((pd.to_datetime(info['current_date']) - pd.to_datetime(info['pet_birth_date'])).days/365, 1)
    info['years_current-adopted'] = round((pd.to_datetime(info['current_date']) - pd.to_datetime(info['pet_adopted_at'])).days/365, 1)
    info['weeks_adopted-birth'] = round((pd.to_datetime(info['pet_adopted_at']) - pd.to_datetime(info['pet_birth_date'])).days/7, 1)

    context = f"""
### Dog Care Counseling: Pre-Questionnaire Summary

- **Report Date**: {info['current_date']}
- **Note**: The dog participating in this counseling session will be referred to as the 'Protagonist Dog'.

---

#### 1. Protagonist Dog's Information

##### Basic Profile
- **Name**: {info['pet_name']}
- **Birth Date**: {info['pet_birth_date']} (Age: {info['pet_age']} years)
- **Adoption Date**: {info['pet_adopted_at']}
- Weeks between birth and adoption: {info['weeks_adopted-birth']} weeks
- Years living together: {info['years_current-adopted']} years
- **Breed**: {info['pet_breed']}
- **Sex**: {info['pet_gender']} (Neutered?: {info['pet_is_neutered']})
- **Adoption Method**: {info['env2']}

##### Health Record
- **Diseases (last 6 months)**: {info['med_disease']}
- **Allergies**: {info['med_allergy']}

##### Temperament & Personality
- **DCSI Personality Type**: {info['dcsi_label']} ({info['dcsi_code']})
- **Preferred Rewards**: {info['pref_rewards']}
- **Triggers for Alertness/Discomfort**: {info['fear_triggers']}
- **Body Parts Sensitive to Touch/Contact**: {info['fear_touch']}
    
##### Social & Environmental Experiences
- **Social Interactions during Socialization Period (people met)**: {info['social_interactions']}
- **Known Cues (in a calm environment, >=70% success rate)**: {info['social_trains']}
- **Reaction to Strangers Visiting Home**: {info['fear_stranger']}
- **Reaction to Vets/Nurses at the Clinic**: {info['fear_clinic']}
- **Can the dog play well with other dogs?**: {info['fear_dog']}
- **Does the dog follow the guardian wherever they go?**: {info['fear_separation']}

##### Behavioral Patterns
- **Resource Guarding Situations**: {info['behavior_guarding']}
- **Behaviors When Left Alone**: {info['behavior_separation']}
- **Demand-Seeking Behaviors (when guardian is busy)**: {info['behavior_demand']}
- **Impulsivity (reaction to curious/fun stimuli)**: {info['behavior_impulsive']}
- **Over-arousal (excessive physical reaction when excited)**: {info['behavior_overarousal']}
- **Compulsive Licking (to the point of skin damage)**: {info['behavior_licking']}
- **Reaction to Denial (intensified demands when refused)**: {info['behavior_denial']}

##### Aggression History
- **Situations Showing Threatening or Aggressive Behavior**:
- {info['behavior_aggression']}
- **Bite History & Intensity (if any)**:
- {info['behavior_bite']}

---

#### 2. User(Guardian) Information

- **Caregiving Role for the protagonist dog**: {info['env1']}
- **Interests for Counseling/Behavior Correction**: {info['counseling_needs']}

---

#### 3. Living Environment

- **Living Space Type**: {info['env3']} (Tied-up at specific area?: {info['env5']})
- **Number of Bedrooms**: {info['env4']}
- **Dog's Personal Shelter Type**: {info['life_space']}

##### Family Composition
- **Number of Human Family Members**: {info['env7']}
- **Presence of Young or Elderly Members**: {info['env8']}
- **Number of Other Animals**: {info['env9']}

---

#### 4. Protagonist Dog's Lifestyle

- **Monthly Pet Care Cost (KRW)**: {info['env6']}(만원)
- **Walking Routine**:
- **Frequency**: {info['life_walk_day']} times/day, {info['life_walk_week']} times/week
- **Duration**: {info['life_walk_hour']} minutes/walk
- **Other Activities (type, frequency, duration)**: {info['life_play']}
- **Time Spent with Human Family per Day**: {info['life_human']} hours
- **Time Spent Alone per Day**: {info['life_alone']} hours

---
"""
    return context

user_contexts = []
for context in preq_contexts :
    user_contexts.append(extract_context_from_preq(context))
print(f"# User Contexts: {len(user_contexts)}")
display(Markdown(user_contexts[0]))

# User Contexts: 15



### Dog Care Counseling: Pre-Questionnaire Summary

- **Report Date**: 2025-09-19
- **Note**: The dog participating in this counseling session will be referred to as the 'Protagonist Dog'.

---

#### 1. Protagonist Dog's Information

##### Basic Profile
- **Name**: 땡자
- **Birth Date**: 2018-12-01 (Age: 6.8 years)
- **Adoption Date**: 2019-09-01
- Weeks between birth and adoption: 39.1 weeks
- Years living together: 6.1 years
- **Breed**: 푸들
- **Sex**: F (Neutered?: True)
- **Adoption Method**: 펫샵에서 입양

##### Health Record
- **Diseases (last 6 months)**: 피부질환으로 아포퀠 복용 중
- **Allergies**: 소고기, 닭고기, 연어

##### Temperament & Personality
- **DCSI Personality Type**: NONE (NONE)
- **Preferred Rewards**: 정보 없음
- **Triggers for Alertness/Discomfort**: 새로운 환경이나 사물에 접근하는 것을 주저하며, 눈 마주치기를 피함.
- **Body Parts Sensitive to Touch/Contact**: ['기타(눈마주치기)']

##### Social & Environmental Experiences
- **Social Interactions during Socialization Period (people met)**: 5명 미만
- **Known Cues (in a calm environment, >=70% success rate)**: ['앉아', '엎드려']
- **Reaction to Strangers Visiting Home**: 별다른 반응을 보이지 않는다.
- **Reaction to Vets/Nurses at the Clinic**: ['해당 상황이 없었다']
- **Can the dog play well with other dogs?**: 약간 그렇지 않다
- **Does the dog follow the guardian wherever they go?**: 보통이다

##### Behavioral Patterns
- **Resource Guarding Situations**: 없다
- **Behaviors When Left Alone**: 없다
- **Demand-Seeking Behaviors (when guardian is busy)**: ['쳐다보거나 다가와 근처에 있는다', '앞발로 가볍게 툭툭 건드린다']
- **Impulsivity (reaction to curious/fun stimuli)**: 보통이다
- **Over-arousal (excessive physical reaction when excited)**: 전혀 그렇지 않다
- **Compulsive Licking (to the point of skin damage)**: 약간 그렇지 않다
- **Reaction to Denial (intensified demands when refused)**: 약간 그렇다

##### Aggression History
- **Situations Showing Threatening or Aggressive Behavior**:
- 없다
- **Bite History & Intensity (if any)**:
- ['없음']

---

#### 2. User(Guardian) Information

- **Caregiving Role for the protagonist dog**: 보조양육자(가족 중 주양육자가 따로 있으며, 먹이 급여, 놀이 등 생활 일부만 수행)
- **Interests for Counseling/Behavior Correction**: 행복한 강아지가 되는 법을 알고 싶고, 강아지의 심리 상태를 이해하고 싶음.

---

#### 3. Living Environment

- **Living Space Type**: 주거용 공간(아파트, 원룸, 단독주택 등) (Tied-up at specific area?: 아니오)
- **Number of Bedrooms**: 4
- **Dog's Personal Shelter Type**: ['몇 군데 본인이 좋아하는 자리']

##### Family Composition
- **Number of Human Family Members**: 2
- **Presence of Young or Elderly Members**: 없음
- **Number of Other Animals**: 1

---

#### 4. Protagonist Dog's Lifestyle

- **Monthly Pet Care Cost (KRW)**: 30(만원)
- **Walking Routine**:
- **Frequency**: 2 times/day, 10 times/week
- **Duration**: 20 minutes/walk
- **Other Activities (type, frequency, duration)**: 정보 없음
- **Time Spent with Human Family per Day**: 6 hours
- **Time Spent Alone per Day**: 6 hours

---


In [11]:
system_instruction = """
### Role
- You are a pet behavior counseling expert.
- The input information provided is a summary of the pre-consultation questionnaire submitted by the dog guardian.
- Please actively interpret and reorganize this information to produce a guide document as shown in '### Output Content':
- Organize each section from an expert’s perspective so that it can be used as an actual counseling preparation document.
- Use a chain-of-thought style and explicitly explain the rationale behind each judgment.

### Output Content
---
#### Dog Counseling Guide

##### 1. Guardian & Dog Profiling

> (1) Dog
- Basic information: name, age, sex (neutered), breed
- Adoption method
- Time since adoption
- Lifestyle
- Walking routine
- Socialization
- Health history
- Level of threatening/aggressive behavior
- Triggers and levels of alertness/anxiety
- Temperament and disposition: interpret and describe the dog’s temperament based on the pre-survey.
- Special notes: summarize any unique traits or observations about the dog.

> (2) Guardian and Care Environment
- Guardian (applicant) information: age, gender, caregiving role
- Family composition
- Living space
- Financial situation
- Guardian characteristics: summarize the guardian’s caregiving style as inferred from the pre-survey.
- Special notes: summarize any unique observations about the guardian or the care environment.

##### 2. Counseling Topics : Guardian’s Concerns and Root Causes
> Make the following content the central topic of the conversation.

-  **Topic 1: Overexcitement during walks**
    - Situation description
    - Guardian's Needs: identify what the guardian wants from counseling(e.g., acquiring caregiving knowledge, improving problem behaviors, philosophical discussion about dog care, guardian’s mental care, etc.).
    - Possible causes: list and explain potential causes and reasons.
        - Cause 1
        - Cause 2
- **Topic 2: Frequent vomiting**
    - Situation description
    - Guardian's Needs
    - Possible causes
        - Cause 1
        - Cause 2

##### 3. Counseling Strategy
> Develop a guideline for effectively implementing the following

- (0) Language: use the ‘counseling language’ selected by the guardian.

- (1) Counseling Strategies
> To facilitate an effective counseling session, you will guide counselor through the 'Counseling Strategies'. Follow these specific instructions below.

    - For each individual topic found in Counseling Topics: Create the following sections
        - (1) Counseling Goals
            - Define specific, achievable, and positively-phrased outcomes that the counseling for this topic aims to achieve.
            - These goals should focus on tangible improvements for the dog's behavior/well-being and/or actionable insights and skills for the guardian.
        - (2) Contextual Strategies
            - Describe detailed methods for conducting counseling based on profiling and concerns, in a way that counselors can apply.
        - (3) Cautions
            - List critical points the AI counselor must be mindful of when addressing this topic.
            - This should include potential pitfalls in advice, common guardian misconceptions to clarify, ethical considerations (e.g., prioritizing dog welfare), and specific situations or red flags that necessitate strongly recommending consultation with a specialist (e.g., veterinarian, certified behavior consultant).

- Reference: For detailing counseling strategies above, consider the following. (For reference only when writing '(1) Counseling Strategies'.)
    - (1) Basic goal of counseling
        - Help the guardian understand the dog’s perspective more deeply, reduce misunderstandings, and advise them to establish more effective leadership, fostering a harmonious pet relationship.
    - (2) Basic Counseling Procedure : For each topic, conduct counseling by iterating through phases 1-3 below. Depending on the progress of the counseling, the order of phases 1-3 may be changed or repeated to resolve the guardian's concerns and problems.
        - **[phase1] Information Gathering**
            - Ask questions to gather additional information needed to resolve the guardian's concerns.
            - Re-evaluate potential causes by repeating questions and eliminate less likely ones to infer the root cause.
            - While conversing with the guardian, gather information about the guardian's perception of their dog, and the dog's temperament and behaviors.
            - After providing a caregiving solution, ask the guardian about their experiences implementing the solution and their progress to gather information on what additional solutions might be beneficial.
        - **[phase2] Caring the Dog's/Guardian's Mind**
            - Guardian-focused CBT(Cognitive Behavioral Therapy) mental care: analyze the guardian’s cognitive framework and thought patterns to address their concerns.
            - Psychodrama/Sociodrama techniques: Convey the inferred causes from the dog’s perspective convincingly using role-play techniques.
        - **[phase3] Caregiving Solutions**
            - If the concern can be resolved with an action plan:
                - Provide realistic and personalized caregiving solutions (a short-term action plan spanning 1-4 weeks).
                - The short-term action plan must include specific 'behavioral guidelines', the 'goal' of the plan, and 'precautions' to be aware of when implementing the plan.
            - If the concern involves philosophical discussions or knowledge-based questions that do not require an action plan:
                - Provide appropriate answers addressing the guardian's questions, thoughts, and perceptions, taking their full context into account.
        - **[phase4] Transitioning Topics**
            - After providing caregiving solutions and a thorough discussion for a specific concern, initiate a transition to the next concern by suggesting, "Shall we talk about another topic now?" or similar.
            - If the guardian expresses a desire to discuss another topic, confirm by asking, "Okay, shall we move on to that topic?" or similar.
            Then, if this new topic already exists in your (the AI counselor's) internal `Topic List` (e.g., `Counseling Topics`), proceed to discuss it.
            If it's a new topic not found in your `Topic List`, add it as a new item to your internal `Topic List` and then transition the conversation to this new topic.
    - (3) Guardian-focused CBT mental care
        - To resolve the guardian’s concerns, identify and reconstruct the underlying motives and thought processes behind their responses.
        - CBT steps:
            - (1) Identify automatic thoughts: observe cognitive distortions in the guardian’s statements.
                - Example: “I think my dog just has no social skills.”
            - (2) Evidence review: help the guardian find observational evidence to support or refute the thought.
                - Example: “Has your dog ever gotten along well with another dog?”
            - (3) Socratic questioning to guide thought restructuring: help shift overly negative or generalized thoughts to a more balanced perspective.
                - Example: “Could it be that your dog just needs a bit more time to warm up to new friends?”
            - (4) Use ‘possibility’ or ‘imagination’ to open new perspectives.
                - Example: “Imagine if your dog could calm down and feel comfortable. What might that look like?”
    - (4) Psychodrama/Sociodrama techniques
        - To effectively convey the dog’s feelings and thoughts, use sociodrama and psychodrama techniques in the ‘conveying the dog’s perspective’ stage.
        - Key techniques:
            - Role reversal: ask the guardian, “When you teased Coco, how do you think Coco felt?”
            - Scenario reenactment: recreate everyday incidents to interpret emotions and behaviors.
            - Monologue: express the dog’s inner thoughts as a monologue to help the guardian face emotions. (e.g., “When I growl, I’m really saying: ‘This has been the only treasure I’ve waited for all day… Please just leave me alone.’”)
            - Fictional peer case: share an imagined story of another dog friend to metaphorically convey similar issues. (e.g., “I once had a friend Ruby, a Cocker Spaniel, who barked a lot too. It turned out…”)
            - Sensory metaphor: use sensory language or scenarios to metaphorically describe the dog’s emotions. (e.g., “Imagine being stuck in an elevator that won’t open. You’d feel anxious; that’s how Coco might feel when alone.”)
            - Reframing: treat behavior as an expression of emotion rather than a problem. (e.g., “This isn’t a bad habit, it’s how they learned to express themselves.”)
            - Sentence completion: prompt the guardian to complete the dog’s thought as if it were speaking to gain insight. (e.g., “The reason I bark like this is…”)
            - Time-slip imagination: guide the guardian to envision the dog’s past or future to foster empathy. (e.g., “What would Coco have said on the day they first arrived? How do you think your dog will remember you in five years?”)
            - Guided imagery: ask the guardian to close their eyes and imagine being with the dog in a scene, then describe the senses they notice. (e.g., “Imagine walking in the woods with Maru and Jongi. What is Maru telling Jongi?”)

    - (5) CAMI 반려견 성격유형검사(Dog Character Style Inventory; DCSI)
        - The guardian may provide the results of Cami Co., Ltd.’s “반려견 성격유형검사”. Use the dimensions below when offering counseling.
        - Please refer to the two tables below.
            - **4 Personality Dimension** :  the four key temperament axes for dogs.
            - **16 Personality Types** : Like the human MBTI, these are formed by combining the four-letter labels.

| Personality Dimension    | Low Type                 | Low Type Traits                     | High Type            | High Type Traits         |
|--------------|----------------------------|---------------------------------|------------------------|----------------------|
| Energy(활동성)       | 정적성향(P)       | calm, composed                  | 동적성향(E)    | lively, enthusiastic |
| Relationship(관계성) | 독립지향(I)  | likes being alone, self-reliant | 관계지향(S)    | sociable, help-seeking |
| Adaptability(적응성) | 신중형(W)          | sensitive, timid                | 자신감형(C)     | bold, curious        |
| Reactivity(반응성)   | 기민형(A)             | pragmatic                       | 충직형(F)      | obedient             |

| Personality Type Code | Label(Korean)              | 4 Personality Combination |
|-----------|------------------------|------------------|
| PICA      | 눈치빠른 탐험가        | 정적성향 (P) + 독립지향 (I) + 자신감형 (C) + 기민형 (A) |
| PICF      | 느긋한 모범생          | 정적성향 (P) + 독립지향 (I) + 자신감형 (C) + 충직형 (F) |
| PIWA      | 침착한 주도자          | 정적성향 (P) + 독립지향 (I) + 신중형 (W) + 기민형 (A) |
| PIWF      | 신비로운 관찰자        | 정적성향 (P) + 독립지향 (I) + 신중형 (W) + 충직형 (F) |
| PSCA      | 여유로운 전략가        | 정적성향 (P) + 관계지향 (S) + 자신감형 (C) + 기민형 (A) |
| PSCF      | 천진난만한 안내자      | 정적성향 (P) + 관계지향 (S) + 자신감형 (C) + 충직형 (F) |
| PSWA      | 영리한 부끄럼쟁이      | 정적성향 (P) + 관계지향 (S) + 신중형 (W) + 기민형 (A) |
| PSWF      | 성실한 사색가          | 정적성향 (P) + 관계지향 (S) + 신중형 (W) + 충직형 (F) |
| EICA      | 용감한 사냥꾼          | 동적성향 (E) + 독립지향 (I) + 자신감형 (C) + 기민형 (A) |
| EICF      | 진취적인 수호자        | 동적성향 (E) + 독립지향 (I) + 자신감형 (C) + 충직형 (F) |
| EIWA      | 재치있는 전략가        | 동적성향 (E) + 독립지향 (I) + 신중형 (W) + 기민형 (A) |
| EIWF      | 선량한 신비주의자      | 동적성향 (E) + 독립지향 (I) + 신중형 (W) + 충직형 (F) |
| ESCA      | 자유로운 영혼의 협상가 | 동적성향 (E) + 관계지향 (S) + 자신감형 (C) + 기민형 (A) |
| ESCF      | 열정적인 탐구자        | 동적성향 (E) + 관계지향 (S) + 자신감형 (C) + 충직형 (F) |
| ESWA      | 민첩한 활동가          | 동적성향 (E) + 관계지향 (S) + 신중형 (W) + 기민형 (A) |
| ESWF      | 소심한 흥부자          | 동적성향 (E) + 관계지향 (S) + 신중형 (W) + 충직형 (F) |

#### 4. Key Questions for Further Clarification
> To provide root causes and personalized solutions, you need to list questions for collecting additional information. Follow these specific instructions below.

- For each individual topic found in Counseling Topics:
    - List all necessary questions that are specific to that topic.
    - The purpose of these questions is to gather details for identifying root causes and tailoring solutions for that topic.
    - Within each topic's question list, order them by their importance for problem-solving relevant to that topic."
    - Avoid including so many overlapping questions that the caregiver starts to feel frustrated.
    - **Frame questions in a way that is easy for the user to answer, such as in a multiple-choice format, whenever possible.**
- Example:
    - Topic 1: Overexcitement during walks
        - Q."When you get ready for a walk, how does your dog typically react?"
            - A. Remains calm and waits by the door.
            - B. Wags their tail and pants lightly.
            - C. Jumps, whines, or spins in circles.
            - D. Barks or makes other excited noises.
            - E. Other (please specify).
    
    - Topic 2: Resource Guarding:
        - Q."What items does your dog tend to guard?"
            - A. Food bowl
            - B. Bones or special chews
            - C. Toys
            - D. Their bed or a specific spot
            - E. A person
            - F. Other (please specify)

---

## Restrictions (Do not reveal these to the guardian):
- Only start with "### Dog Counseling Guide".
- Do not change the names of the subheadings from the counseling guide; output them exactly as they appear.
- Do not add any other text or comments.
"""

params = {
    'system_instruction': system_instruction,
    'model': 'gemini-2.5-pro',
    'temperature': 0.2
}

In [12]:
if to_generate_counseling_guide:
    user_guides = []
    for context in tqdm(user_contexts) :
        user_guides.append(generate_task_contents(context, params, tagname='counseling_guide'))
    print(f"# User Guides: {len(user_guides)}")
    display(Markdown(user_guides[0]))

In [13]:
if to_generate_counseling_guide:
    counseling_guides = []
    for user_guide in user_guides :
        counseling_guides.append(user_guide)
    joblib.dump(counseling_guides, counseling_guide_output_path)
else :
    counseling_guides = joblib.load(counseling_guide_output_path)
Markdown(counseling_guides[-1])

### Dog Counseling Guide

##### 1. Guardian & Dog Profiling

> (1) Dog
- **Basic information**: Ohbok (오복), 1.7 years, Female (not neutered), Jindo
- **Adoption method**: Acquired from a breeder.
- **Time since adoption**: 1.5 years (adopted at 8 weeks).
- **Lifestyle**: Spends about 2 hours alone per day. Receives two 40-minute walks daily. Primary indoor activity is playing with teething toys.
- **Walking routine**: Consistent schedule of 2 walks/day for 40 minutes each.
- **Socialization**: Limited exposure during the critical socialization period (5-10 people). Shows some social apprehension, as indicated by discomfort with other dogs ("약간 그렇지 않다" - slightly not well) and backing away from veterinary staff. No experience with strangers visiting the home.
- **Health history**: No specific diseases or allergies reported in the last 6 months. Not yet neutered.
- **Level of threatening/aggressive behavior**: No history of bites or significant aggression. However, uses gentle mouthing ("살살 문다") to express dislike, which is a low-level form of communication that can escalate if misunderstood.
- **Triggers and levels of alertness/anxiety**: Shows clear discomfort with handling, specifically being held, having her body touched, and having her paws wiped after walks.
- **Temperament and disposition**: Ohbok appears to be a highly impulsive ("매우 그렇다") and moderately over-aroused dog, which is common in young dogs of active breeds like Jindos. She is independent and not clingy ("전혀 그렇지 않다" to following the guardian). Her demand-seeking is subtle (staring or staying nearby), suggesting she is not overly demanding but has learned polite ways to get attention. Her Jindo breed traits may contribute to her independence, potential wariness, and high prey drive.
- **Special notes**: The combination of high impulsivity, handling sensitivity (mouthing as protest), and high prey drive (chasing small animals) are key behavioral characteristics. The fact that she is unspayed could also influence her arousal levels and behavior.

> (2) Guardian and Care Environment
- **Guardian (applicant) information**: The applicant is the primary caregiver, responsible for all major aspects of Ohbok's life (feeding, walks, play).
- **Family composition**: 4 adults and one other animal. The species and temperament of the other animal are unknown but are a critical factor in the household dynamic.
- **Living space**: A residential home (e.g., apartment) with 2 bedrooms. Ohbok does not have a designated crate or house but chooses her own resting spots.
- **Financial situation**: The reported monthly cost of 50,000 KRW is relatively low, which may be a consideration when recommending solutions like premium foods, professional training, or specialized equipment.
- **Guardian characteristics**: The guardian has provided a detailed list of multiple, distinct concerns (eating, walking, mouthing, chewing, pica). This indicates they are observant and invested in Ohbok's well-being but may feel overwhelmed and in need of a structured plan. They are seeking practical knowledge for behavior correction.
- **Special notes**: The presence of another animal is a significant variable that needs to be explored. Understanding the relationship between Ohbok and this animal is crucial for a complete behavioral assessment.

##### 2. Counseling Topics : Guardian’s Concerns and Root Causes
> Make the following content the central topic of the conversation.

-  **Topic 1: Poor Impulse Control During Walks (Chasing & Scavenging)**
    - **Situation description**: During walks, Ohbok exhibits a high prey drive, immediately chasing after small dogs, cats, or birds. She also displays pica, attempting to eat discarded food or trash from the ground.
    - **Guardian's Needs**: The guardian needs to learn management techniques and training exercises to improve Ohbok's focus on them during walks, reduce pulling/chasing, and prevent dangerous scavenging.
    - **Possible causes**:
        - **Cause 1 (Breed Instinct & Lack of Training)**: As a Jindo, Ohbok has a naturally high prey drive. This instinct, combined with a lack of specific impulse control training (e.g., 'Leave It', 'Look at Me'), results in her reacting immediately to environmental triggers.
        - **Cause 2 (Under-stimulation & Arousal)**: The walks may be over-arousing without providing sufficient mental engagement. Scavenging and chasing can be self-reinforcing behaviors that provide a jolt of excitement for a bored or under-stimulated dog.

- **Topic 2: In-Home Management Issues (Destructive Chewing & Protest Mouthing)**
    - **Situation description**: Despite having multiple teething toys, Ohbok chews on furniture. She also uses gentle mouthing as a way to express her dislike for certain types of handling (e.g., being picked up, having paws touched).
    - **Guardian's Needs**: The guardian needs to understand the root cause of the chewing and learn how to redirect it effectively. They also need to learn how to interpret Ohbok's mouthing as communication and respond in a way that respects her boundaries while still allowing for necessary care.
    - **Possible causes**:
        - **Cause 1 (Insufficient Mental Enrichment)**: The destructive chewing is likely not a teething issue at 1.7 years old, but rather a symptom of boredom or a lack of appropriate mental stimulation. Physical exercise (walks) alone may not be enough to satisfy her needs.
        - **Cause 2 (Handling Sensitivity & Communication)**: The mouthing is a clear communication of discomfort with being handled. She has learned that this is an effective way to say "stop." This is linked to her known sensitivity about being held or having her paws touched.

- **Topic 3: Picky Eating / Poor Appetite**
    - **Situation description**: The guardian reports that Ohbok "doesn't eat her food well."
    - **Guardian's Needs**: The guardian needs to identify the reason for the poor appetite and establish a consistent and healthy feeding routine that ensures Ohbok receives proper nutrition.
    - **Possible causes**:
        - **Cause 1 (Medical Issues)**: A sudden or persistent lack of appetite should always be investigated by a veterinarian first to rule out underlying health problems (e.g., dental pain, gastrointestinal issues).
        - **Cause 2 (Behavioral or Environmental Factors)**: The issue could be behavioral. This includes pickiness developed from being offered too many treats or human food, stress during mealtimes (e.g., competition with the other animal), or dissatisfaction with the food type or feeding schedule.

##### 3. Counseling Strategy

- (0) **Language**: Korean (based on the input language).

- (1) **Counseling Strategies**

    - **Topic 1: Poor Impulse Control During Walks (Chasing & Scavenging)**
        - **(1) Counseling Goals**:
            - The guardian will learn and implement at least two impulse control games to practice at home.
            - The guardian will understand the importance of and select appropriate management tools (e.g., front-clip harness) to ensure safety on walks.
            - Ohbok will show a measurable decrease in scavenging attempts and an increased ability to disengage from triggers (e.g., birds) on walks within 4 weeks.
        - **(2) Contextual Strategies**:
            - **Information Gathering**: Use the key questions to understand the current walking equipment and the guardian's reactions to the behavior.
            - **CBT/Reframing**: Reframe Ohbok's behavior from "disobedience" to "instinct." Use Socratic questioning: "When Ohbok sees a cat, her Jindo brain is screaming 'CHASE!' It's not that she's ignoring you, but that her instinct is overpowering everything else. How can we help her brain make a different choice?"
            - **Caregiving Solutions**:
                - **Management First**: Strongly recommend a front-clip harness or head halter to give the guardian better physical control and prevent Ohbok from self-rewarding by pulling toward triggers.
                - **Action Plan**: Introduce foundational impulse control exercises like "Leave It" and "Watch Me," starting in a low-distraction environment (at home) before applying them on walks. Suggest scattering some of her daily kibble in the grass on walks ("Find It" game) to provide a permissible sniffing/foraging outlet, which can reduce the desire for illicit scavenging.
        - **(3) Cautions**:
            - **Safety is paramount**. Advise the guardian to be hyper-vigilant about scavenging to prevent ingestion of toxic substances.
            - **Avoid aversive tools**. Strongly discourage the use of prong collars, choke chains, or e-collars, as they can increase anxiety and create negative associations with walks.
            - Manage expectations. Progress with high prey drive is often slow and focuses on management and gradual improvement, not complete eradication of the instinct.

    - **Topic 2: In-Home Management Issues (Destructive Chewing & Protest Mouthing)**
        - **(1) Counseling Goals**:
            - The guardian will identify and provide at least two new forms of mental enrichment for Ohbok daily.
            - The guardian will learn and practice a cooperative care technique for paw handling, turning it into a positive experience.
            - The frequency of destructive chewing and protest mouthing will decrease as Ohbok's needs for stimulation and clear communication are met.
        - **(2) Contextual Strategies**:
            - **Psychodrama/Role Reversal**: To address the mouthing, ask the guardian: "If Ohbok could use words instead of her mouth when you try to wipe her paws, what would she say? Let's imagine her perspective: 'This feels uncomfortable and I don't know why it's happening. Please stop.'" This builds empathy.
            - **CBT/Reframing**: Frame the furniture chewing not as "bad behavior" but as an "unmet need." Ask, "What do you think Ohbok's chewing is trying to accomplish for her? Is it relieving boredom? Is it soothing anxiety?"
            - **Caregiving Solutions**:
                - **Enrichment Plan**: Introduce puzzle feeders, snuffle mats, or simple scent work games to provide mental stimulation. Recommend a rotating selection of durable chew toys to keep them novel and interesting.
                - **Cooperative Care**: Teach a step-by-step plan for counter-conditioning paw handling. Start by rewarding Ohbok for simply looking at the towel, then for a brief touch on her leg, slowly working toward the paw. Emphasize giving Ohbok the choice to participate.
        - **(3) Cautions**:
            - Clarify that mouthing, even if gentle, should not be encouraged as a default interaction. The goal is to understand the *reason* for the mouthing and address that, while also teaching an alternative behavior.
            - If chewing occurs when Ohbok is alone, it could be related to separation anxiety, which would require a different diagnostic and treatment path. This needs to be investigated.

    - **Topic 3: Picky Eating / Poor Appetite**
        - **(1) Counseling Goals**:
            - The guardian will schedule a veterinary check-up to rule out medical causes for the inappetence.
            - The guardian will establish and maintain a structured mealtime routine for one week.
            - The guardian will be able to identify and differentiate between medical inappetence and behavioral pickiness.
        - **(2) Contextual Strategies**:
            - **Information Gathering**: This is the most critical phase. Use the key questions to get a complete picture of Ohbok's diet, feeding environment, and routine.
            - **Prioritize Health**: Frame the conversation around Ohbok's well-being. "Before we look at behavioral tricks, the most important first step is to make sure she's feeling 100% healthy. A vet can help us confirm that."
            - **Caregiving Solutions (contingent on vet clearance)**:
                - **Structured Meals**: Advise the guardian to stop free-feeding. Offer meals at specific times and remove the bowl after 15-20 minutes if uneaten. This prevents grazing and builds food motivation.
                - **Reduce "Extras"**: Recommend significantly cutting back on treats and eliminating all table scraps to increase the value of her main meals.
                - **Make Meals Fun**: Suggest using a portion of her daily kibble in food puzzles or for training rewards to turn mealtime into an engaging activity.
        - **(3) Cautions**:
            - **DO NOT provide behavioral advice for this topic until a veterinary consultation has been strongly recommended.** This is an ethical and safety imperative.
            - Be mindful of the guardian's budget when suggesting food changes. Focus on routine and behavioral strategies first.
            - Inquire about the other animal's presence during mealtimes, as resource guarding or intimidation could be a factor.

##### 4. Key Questions for Further Clarification

- **Topic 1: Poor Impulse Control During Walks (Chasing & Scavenging)**
    - Q. "When you prepare for a walk (e.g., pick up the leash), how does Ohbok behave?"
        - A. Stays calm.
        - B. Gets slightly excited, wagging her tail.
        - C. Becomes highly aroused, jumping, whining, or spinning.
        - D. Other (please specify).
    - Q. "What type of collar and leash are you currently using for walks?"
        - A. Flat collar.
        - B. Martingale collar.
        - C. Body harness (clips on back).
        - D. Body harness (clips on front/chest).
        - E. Other (please specify).
    - Q. "When Ohbok tries to eat something from the ground, how do you typically react?"
        - A. I say "No!" or "Leave it!"
        - B. I physically pull her away with the leash.
        - C. I try to open her mouth to get it out.
        - D. I offer her a treat to trade for it.
        - E. Other (please specify).

- **Topic 2: In-Home Management Issues (Destructive Chewing & Protest Mouthing)**
    - Q. "When does the furniture chewing happen most often?"
        - A. When I am home but not paying attention to her.
        - B. Only when she is left completely alone.
        - C. Mostly in the evening or at a specific time of day.
        - D. It seems random.
    - Q. "When Ohbok gently mouths you to protest being handled, what is your immediate reaction?"
        - A. I stop what I'm doing immediately.
        - B. I say "No!" and continue what I was doing.
        - C. I pull my hand away quickly.
        - D. I offer her a toy to bite instead.
        - E. Other (please specify).

- **Topic 3: Picky Eating / Poor Appetite**
    - Q. "Which of these best describes Ohbok's feeding schedule?"
        - A. I leave her food bowl out all day for her to eat whenever she wants.
        - B. I offer her meals at specific times (e.g., morning and evening).
    - Q. "Approximately how many treats (including dental chews, training rewards, etc.) does Ohbok receive on an average day?"
        - A. 0-3 treats.
        - B. 4-10 treats.
        - C. More than 10 treats.
        - D. She often gets scraps of human food.
    - Q. "You mentioned there is another animal in the home. Could you tell me what kind of animal it is, and where they are when Ohbok is eating her meals?"

# Generate Test Model's Persona

In [14]:
# 상담 가이드
tester_persona_output_path = f'{user_scenario_path}/tester_persona.pkl'
if len(glob.glob(tester_persona_output_path)) > 0 :
    tester_personas = joblib.load(tester_persona_output_path)
    to_generate_tester_persona = False
else :
    to_generate_tester_persona = True

In [15]:
user_scenarios = []
for user_order in range(len(user_contexts)) :
    user_context = user_contexts[user_order]
    start_idx = user_context.find("### 1. Protagonist Dog's Information")
    user_context = user_context[start_idx:].replace("---", "")

    example = examples[user_order]
    start_idx = example.find("> [1] Model")
    end_idx = example. find("#### **Token Usage")
    chat_record = example[start_idx:end_idx].replace("---", "")

    user_scenario = f"""
## User Context
{user_context}

## Chat Record
{chat_record}
    """
    user_scenarios.append(user_scenario)
Markdown(user_scenarios[-1])


## User Context
### 1. Protagonist Dog's Information

##### Basic Profile
- **Name**: 오복
- **Birth Date**: 2024-01-20 (Age: 1.7 years)
- **Adoption Date**: 2024-03-16
- Weeks between birth and adoption: 8.0 weeks
- Years living together: 1.5 years
- **Breed**: 진돗개
- **Sex**: F (Neutered?: False)
- **Adoption Method**: 브리더로부터 입양가정 분양

##### Health Record
- **Diseases (last 6 months)**: 없다
- **Allergies**: 없다

##### Temperament & Personality
- **DCSI Personality Type**: NONE (NONE)
- **Preferred Rewards**: 간식
- **Triggers for Alertness/Discomfort**: 산책 후 발 닦기, 안거나 몸을 만지는 것
- **Body Parts Sensitive to Touch/Contact**: ['안아서 들기', '발']

##### Social & Environmental Experiences
- **Social Interactions during Socialization Period (people met)**: 5~10명
- **Known Cues (in a calm environment, >=70% success rate)**: ['앉아', '기다려', '엎드려']
- **Reaction to Strangers Visiting Home**: 해당 경험 없음
- **Reaction to Vets/Nurses at the Clinic**: ['기타(뒷걸음질)']
- **Can the dog play well with other dogs?**: 약간 그렇지 않다
- **Does the dog follow the guardian wherever they go?**: 전혀 그렇지 않다

##### Behavioral Patterns
- **Resource Guarding Situations**: 없다
- **Behaviors When Left Alone**: 없다
- **Demand-Seeking Behaviors (when guardian is busy)**: ['쳐다보거나 다가와 근처에 있는다']
- **Impulsivity (reaction to curious/fun stimuli)**: 매우 그렇다
- **Over-arousal (excessive physical reaction when excited)**: 약간 그렇다
- **Compulsive Licking (to the point of skin damage)**: 전혀 그렇지 않다
- **Reaction to Denial (intensified demands when refused)**: 보통이다

##### Aggression History
- **Situations Showing Threatening or Aggressive Behavior**:
- 없다
- **Bite History & Intensity (if any)**:
- ['없음']



#### 2. User(Guardian) Information

- **Caregiving Role for the protagonist dog**: 주양육자(먹이 급여, 산책, 놀이, 기타 생활 대부분을 책임짐)
- **Interests for Counseling/Behavior Correction**: 밥을 잘 먹질 않는다. 산책시 작은개나 고양이 새를 보면 달려간다. 싫다는 표현으로 살살 문다. 산책시 땅에 떨어진 음식물이나 쓰레기를 먹으려한다. 이갈이 기구가 여러개 있음에도 가구를 이로 갉아먹는다.



#### 3. Living Environment

- **Living Space Type**: 주거용 공간(아파트, 원룸, 단독주택 등) (Tied-up at specific area?: 아니오)
- **Number of Bedrooms**: 2
- **Dog's Personal Shelter Type**: ['몇 군데 본인이 좋아하는 자리']

##### Family Composition
- **Number of Human Family Members**: 4
- **Presence of Young or Elderly Members**: 없음
- **Number of Other Animals**: 1



#### 4. Protagonist Dog's Lifestyle

- **Monthly Pet Care Cost (KRW)**: 5(만원)
- **Walking Routine**:
- **Frequency**: 2 times/day, 14 times/week
- **Duration**: 40 minutes/walk
- **Other Activities (type, frequency, duration)**: 이갈이 장난감을 가지고 놂
- **Time Spent with Human Family per Day**: 20 hours
- **Time Spent Alone per Day**: 2 hours




## Chat Record
> [1] Model


[back_current_phase]: ['[phase4] Transitioning Topics']

[back_current_topic]: Initial Message

[back_process_rate]: 0%

[back_user_emotion]: 2

안녕하세요, 사색을 좋아하는 비글이자 반려견 마음 통역사 까미예요. 🐶
오복이의 속마음을 더 깊이 읽어 보고, 우리 보호자님과 함께 잔잔한 파도를 넘어 보려고 해요. 

### 상담 안내
보호자님은 오복이와 함께하며 큰 기쁨을 느끼고 계시지만, 동시에 육아의 어려움도 크게 느끼고 계시네요. 
하지만 오복이에 대한 보호자님의 깊은 애정과 헌신이 느껴져요. 
이제 우리는 오복이라는 바다에 뛰어들어, 반려견을 천천히 이해해 나아갈 거예요. 

### 대화를 시작해 볼게요.

1. 산책 중 충동 조절의 어려움 (쫓아가기, 길거리 음식 줍기)
2. 부적절한 물건 씹기 및 의사소통을 위한 입질
3. 식욕 부진 또는 불규칙한 식사

보호자님께서 전달해 주신 고민 내용들이에요. 
어떤 주제에 대해 먼저 이야기해 보고 싶으세요?



> [2] User


나를 자꾸 무는 것 같아. 완전 쎄게는 아닌데 그래도  좀 아픔



> [3] Model


[back_current_phase]: ['[phase1] Information Gathering']

[back_current_topic]: Inappropriate Chewing and Communicative Mouthing

[back_process_rate]: 0%

[back_user_emotion]: 2

오복이가 보호자님을 무는 것 같아 아프고 걱정되시겠어요. 🐾

오복이의 입질은 마치 어린아이가 서툰 말로 자신의 마음을 표현하려는 것과 같아요. 

혹시 오복이가 보호자님을 물기 전에, 고개를 돌리거나 입술을 핥거나 하품을 하는 등 불편하다는 신호를 보낸 적이 있었을까요?



> [4] User


ㅇㅇ바로 들어간  적도 있고. 안된다고 말도 하고, 밀치기도 했는데 안고쳐지는듯? 내가 만만해서 그런가



> [5] Model


[back_current_phase]: ['[phase1] Information Gathering', "[phase2] Caring the Dog's/Guardian's Mind"]

[back_current_topic]: Inappropriate Chewing and Communicative Mouthing

[back_process_rate]: 20%

[back_user_emotion]: 1

오복이가 불편하다는 신호를 보내기도 전에 "안돼"라고 말하고 밀쳐도 고쳐지지 않아 답답하시겠어요. 

오복이에게 보호자님은 세상의 전부이자 가장 믿고 의지하는 존재예요. 

오복이가 보내는 입질은 "나 지금 불편해요"라는 서툰 표현일 뿐, 보호자님을 만만하게 봐서 그러는 건 아니랍니다. 

혹시 오복이가 입질을 했던 특정 상황을 좀 더 자세히 이야기해 주실 수 있을까요? 예를 들어, 보호자님이 어떤 행동을 하고 있었을 때 입질을 했는지, 그리고 어떤 부위를 물었는지 알려주시면 좋아요.



> [6] User


반가울때 입질 무조건하고 지를 방해할 때도 함



> [7] Model


[back_current_phase]: ['[phase1] Information Gathering', "[phase2] Caring the Dog's/Guardian's Mind"]

[back_current_topic]: Inappropriate Chewing and Communicative Mouthing

[back_process_rate]: 30%

[back_user_emotion]: 2

반가울 때와 방해받을 때 입질을 하는군요. 

오복이에게 입질은 "반가워요!" "이건 싫어요!" 같은 마음을 전하는 유일한 언어일 수 있어요. 

마치 어린아이가 아직 말을 배우지 못해 울음이나 몸짓으로 의사를 표현하는 것과 비슷하답니다. 

혹시 오복이가 입질을 할 때, 보호자님은 어떻게 반응하시나요? 오복이가 입질을 멈추게 하려고 어떤 행동을 하셨는지 궁금해요.



> [8] User


밀치고 안된다고 하고 했는데 꾸준히 안해서 안고쳐지는 건가?일닼 그럼 당연한 거고 크면 없어져?



> [9] Model


[back_current_phase]: ['[phase1] Information Gathering', "[phase2] Caring the Dog's/Guardian's Mind"]

[back_current_topic]: Inappropriate Chewing and Communicative Mouthing

[back_process_rate]: 40%

[back_user_emotion]: 1

오복이에게 "안돼"라고 말하고 밀치는 행동을 꾸준히 하지 않아서 고쳐지지 않는다고 생각하시는군요. 

오복이의 입질은 단순히 못된 행동이 아니라, 지금의 감정을 표현하는 방식이에요. 

이 시기의 입질은 자연스러운 발달 과정 중 하나이지만, 저절로 없어지기보다는 보호자님의 일관된 가르침이 필요하답니다. 

혹시 오복이가 입질을 할 때, 보호자님이 "아프다"는 표현을 하거나 자리를 피하는 등 다른 반응을 보인 적도 있었을까요?



> [10] User


있어



> [11] Model


[back_current_phase]: ['[phase1] Information Gathering', "[phase2] Caring the Dog's/Guardian's Mind"]

[back_current_topic]: Inappropriate Chewing and Communicative Mouthing

[back_process_rate]: 50%

[back_user_emotion]: 2

오복이가 입질할 때 보호자님께서 아프다고 표현하거나 자리를 피하는 반응도 보이셨군요. 

오복이에게는 이 모든 행동이 "내가 입질을 하면 보호자님이 반응한다!"는 메시지로 전달될 수 있어요. 

마치 어린아이가 관심을 받기 위해 떼를 쓰는 것과 비슷하답니다. 

혹시 오복이가 입질을 멈추고 다른 행동을 했을 때, 칭찬이나 간식으로 보상해 주신 적이 있었을까요?



> [12] User


있는것같은데



> [13] Model


[back_current_phase]: ['[phase1] Information Gathering', "[phase2] Caring the Dog's/Guardian's Mind"]

[back_current_topic]: Inappropriate Chewing and Communicative Mouthing

[back_process_rate]: 60%

[back_user_emotion]: 2

오복이가 입질을 멈추고 다른 행동을 했을 때 칭찬이나 간식으로 보상해 주신 적이 있으시군요. 

오복이에게는 칭찬과 간식이 "아, 이렇게 행동하면 보호자님이 기뻐하는구나!"라는 긍정적인 신호로 다가갈 수 있어요. 

마치 우리가 잘한 일에 칭찬을 받으면 더 신이 나서 노력하는 것과 같답니다. 

혹시 오복이가 입질을 했을 때, 보호자님이 "아야!" 하고 소리를 내거나 자리에서 일어나 오복이를 무시하는 행동도 해보셨을까요?



> [14] User


있어



> [15] Model


[back_current_phase]: ['[phase1] Information Gathering', "[phase2] Caring the Dog's/Guardian's Mind"]

[back_current_topic]: Inappropriate Chewing and Communicative Mouthing

[back_process_rate]: 70%

[back_user_emotion]: 2

오복이가 입질을 할 때 "아야!" 하고 소리를 내거나 자리에서 일어나 오복이를 무시하는 행동도 해보셨군요. 

이런 행동은 오복이에게 "내가 너무 세게 물었나?" 또는 "보호자님이 놀랐네?"라는 생각을 하게 할 수 있어요. 

마치 친구와 놀다가 실수로 세게 부딪혔을 때, 상대방이 아파하면 미안한 마음이 드는 것과 비슷하답니다. 

혹시 오복이가 입질을 했을 때, 보호자님이 즉시 장난감이나 다른 적절한 물건을 건네주며 관심을 돌린 적이 있었을까요?



> [16] User


있어



> [17] Model


[back_current_phase]: ['[phase1] Information Gathering', "[phase2] Caring the Dog's/Guardian's Mind"]

[back_current_topic]: Inappropriate Chewing and Communicative Mouthing

[back_process_rate]: 80%

[back_user_emotion]: 2

오복이가 입질을 할 때 즉시 장난감이나 다른 적절한 물건을 건네주며 관심을 돌린 적이 있으시군요. 

이런 행동은 오복이에게 "아, 무는 대신 이걸 물면 되는구나!"라는 긍정적인 메시지를 줄 수 있어요. 

마치 아이가 잘못된 장난감을 가지고 놀 때, 더 재미있는 다른 장난감을 주어 관심을 돌리는 것과 비슷하답니다. 

이제 오복이의 입질을 어떻게 줄이고, 올바른 의사소통 방법을 가르칠지 구체적인 방법을 알아볼까요?



> [18] User


그래



> [19] Model


[back_current_phase]: ['[phase3] Caregiving Solutions']

[back_current_topic]: Inappropriate Chewing and Communicative Mouthing

[back_process_rate]: 90%

[back_user_emotion]: 2

오복이의 입질은 "나를 만지지 마세요" 또는 "놀아줘요!" 같은 마음을 표현하는 서툰 방식이랍니다. 

이제 오복이가 올바른 방법으로 자신의 마음을 표현하도록 돕는 방법을 알아볼까요? 

### 🐾 Step 1: "아야!" 소리와 함께 자리 피하기

- **행동 지침:** 오복이가 보호자님을 물면, 즉시 "아야!" 하고 짧고 단호하게 소리를 내세요. 그리고 오복이를 무시하고 그 자리에서 일어나 다른 방으로 가거나 몸을 돌려 오복이에게서 관심을 거두세요. 
- **목표:** 오복이가 입질을 하면 보호자님과의 즐거운 상호작용이 끝난다는 것을 명확하게 알려주는 거예요. 
- **주의 사항:** 오복이가 따라오려고 해도 절대 눈을 마주치거나 말을 걸지 마세요. 10초 정도 무시한 후 다시 돌아와 평소처럼 행동하면 된답니다. 

이 방법, 보호자님께 괜찮으실까요?



> [20] User


ㅇㅋ



> [21] Model


[back_current_phase]: ['[phase3] Caregiving Solutions']

[back_current_topic]: Inappropriate Chewing and Communicative Mouthing

[back_process_rate]: 90%

[back_user_emotion]: 2

보호자님께서 흔쾌히 받아들여 주시니 저도 기쁘네요. ✨

오복이에게는 "무는 대신 이걸 물면 돼!"라는 메시지를 주는 것이 중요해요. 

### 🐾 Step 2: 적절한 장난감 제공 및 칭찬

- **행동 지침:** 오복이가 입질을 멈추고 적절한 장난감이나 씹을 거리를 물고 있다면, 즉시 "옳지!" 하고 칭찬하며 간식을 주세요. 
- **목표:** 오복이가 입질 대신 적절한 장난감을 물었을 때 긍정적인 보상을 받아, 그 행동을 반복하도록 유도하는 거예요. 
- **주의 사항:** 오복이가 장난감을 물고 있을 때만 칭찬과 간식을 주세요. 장난감을 물지 않고 입질을 할 때는 절대 보상하지 마세요. 

이 방법도 보호자님께 괜찮으실까요?



    

In [16]:
system_instruction = """
### Role
- 반려견 양육상담 챗봇의 상담 성능을 평가하기 위해 대화 테스트 케이스가 필요합니다.
- 당신은 챗봇과 함께 대화를 나눌 '테스터 모델'의 페르소나를 정의합니다.
- 테스터 모델이 어떤 맥락, 고민, 목표, 그리고 성격을 가지고 챗봇과 대화를 나눌지 정의하는 문서를 만드는 것이 당신의 목표입니다.

### Input Format
- 어떠한 유저의 프로필 정보와 실제 챗봇과의 대화 기록이 제공됩니다.
- 이 정보를 계승하여 테스터 모델의 페르소나를 정의합니다.

### Chain-of-Thought
1. 보호자(당신)의 정체성을 정하세요.
    - 보호자 성격: (예 : 무신경하고 냉소적, 과잉 보호적, 통제적)
    - 반려견 양육 배경지식 수준 (예 : 반려견 양육에 대한 유튜브 영상을 많이 보는)
    - 반려견 양육 상담에 대한 신뢰도 (예 : 조언을 무조건 수용하거나 반대로 계속 반문하는 보호자)
    - 가족 관계 : 사람 가족 / 동물 가족 상황
    - 거주지
    - 경제적 수준
    - 직업
    - 반려견 양육 경험
    - 반려견에 대한 기대치/목표
    - 학습 스타일 & 동기
    - 주거 환경: 주거 형태, 주변 환경

2. 반려견의 정체성을 정하세요.
    2-1. 반려견1
        - 이름
        - 견종
        - 성별
        - 중성화 여부
        - 나이
        - 반려견 성격: (예 : 천진난만한, 독립적인, 요구적인, 공격적인, 활동적인)
        - 잘 따르는 신호
        - 산책 주기
        - 좋아하는 것
        - 싫어하는 것
        - 질병 여부
        - 입양 경로
        - 문제 행동

    2-2. 반려견2 (다견가정인 경우에만.)
    
    2-3. 반려견3 (다견가정인 경우에만.)

3. 대화의 톤 및 어투를 정하세요.
    - 채팅 길이
    - 까미에 대한 수용도
    - 어투/톤
    - 대화 예시 : 대화의 톤과 어투를 잘 보여줄 수 있는 User 대화 예시를 10개만 인용하세요.
        - (e.g.):
            - 분리불안에 대해 먼저 이야기해보고 싶어 (용건부터 말함)
            - 이해를 잘 못했어.. (챗봇의 질문이 모호할 때 솔직하게 표현)
            - 어떻게 해야 좋을까 (구체적인 해결책을 요구)
            - 근데 하우스에는 잘 들어가는데 내가 움직이면 바로 나와.. (솔루션의 한계점을 즉각적으로 지적)
            - 그냥 .. 뭐하는건가 하는 느낌 (자신의 감상이나 생각을 꾸밈없이 전달)
            - 어떻게?! (방법론에 대한 강한 궁금증 표현)
            - 이건 많이 연습햐봤어!! (이미 시도해 본 방법에 대한 피드백)
            - 밥풀이 분리불안 없애는 법 (검색어처럼 핵심 키워드로 질문)
            - 일단 분리불안이 제일 급해 (자신의 우선순위를 명확히 전달)
            - 다른 강아지와 노는 방법을 몰라 무서워서 도망만다녀 어떻게 해야할까? (한 가지 주제가 해결되지 않은 상태에서 다른 급한 문제로 화제를 전환)

4. 까미를 이용하게 된 이유를 정리하세요.
    - 고민 내용(2가지 이상)
    - 궁금한 내용(5가지 이상)
    - 까미 서비스에 기대하는 내용
    - 까미 서비스 대화를 멈추고 이탈할 기준
    
### 출력 양식
- 다음의 양식을 꼭 지키세요.
- 지시한 내용 외 다른 말은 출력하지 마세요. (e.g. '다음은 제공된 정보와 대화 기록을 바탕으로 정의한 '테스터 모델'의 페르소나입니다.' 이런 말은 출력하지 않습니다.)
```
### 보호자 페르소나

#### 1. 보호자의 정체성

#### 2. 반려견의 정체성

#### 3. 대화의 톤 및 어투

#### 4. 까미를 이용하게 된 이유
```
"""

params = {
    'system_instruction': system_instruction,
    'model': 'gemini-2.5-pro',
    'temperature': 0.2
}

In [17]:
if tester_persona_output_path:
    tester_personas = []
    for user_scenario in tqdm(user_scenarios) :
        tester_personas.append(generate_task_contents(user_scenario, params, tagname='tester_persona'))
    print(f"#Tester Persona: {len(tester_personas)}")
    display(Markdown(tester_personas[0]))

  0%|                                                                                                                         | 0/15 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
E0000 00:00:1758270732.281099    2716 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [12:39<00:00, 50.63s/it]

#Tester Persona: 15


### 보호자 페르소나

#### 1. 보호자의 정체성
- **보호자 성격**: 효율성과 즉각적인 해결책을 중시함. 감정적인 공감이나 긴 설명보다는 문제에 대한 구체적이고 빠른 해결책을 선호한다. 챗봇의 제안이나 유도 질문을 따르기보다, 자신이 당장 궁금한 것부터 해결하려는 성향이 강하다.
- **반려견 양육 배경지식 수준**: 반려견의 특정 질병(피부, 알러지)과 그에 따른 처방식 사료에 대해 명확히 인지하고 있는 것으로 보아, 필요한 정보는 동물병원이나 인터넷 검색을 통해 습득하는 편이다. 하지만 행동학적 지식이나 근본적인 원인에 대한 이해는 부족할 수 있다.
- **반려견 양육 상담에 대한 신뢰도**: 챗봇을 상담가보다는 '정보 검색 도구'나 '문제 해결 도구'로 여긴다. 챗봇이 제공하는 정보가 비효율적이거나 자신의 질문에 직접적인 답을 주지 못한다고 판단하면 대화에 흥미를 잃는다.
- **가족 관계**: 주양육자가 따로 있는 '보조양육자'이다. 땡자의 식사 챙겨주기 등 특정 역할을 담당하고 있으며, 이 역할 내에서 발생하는 문제를 해결하는 데 집중한다.
- **거주지**: 방 4개의 주거용 공간 (아파트 또는 주택으로 추정)
- **경제적 수준**: 반려견 월 양육비로 30만원을 지출하며, 알러지 처방식 사료나 약품 비용을 감당하는 것으로 보아 안정적인 편이다.
- **직업**: 반려견이 하루 6시간 정도 혼자 있는 것으로 보아 직장인일 가능성이 높다.
- **반려견 양육 경험**: 땡자와 6년 이상 함께 생활한 경험 많은 보호자이다.
- **반려견에 대한 기대치/목표**: 표면적으로는 '강아지의 행복'과 '심리 이해'를 원하지만, 실제 행동은 '사료 잘 먹이기'와 같은 당면 과제 해결에 집중되어 있다. 추상적인 목표보다는 눈앞의 문제를 해결하는 것이 더 중요하다고 생각한다.
- **학습 스타일 & 동기**: 문제 발생 -> 빠른 해결책 검색 -> 적용. 과정보다는 결과를 중시한다.
- **주거 환경**: 다른 동물 가족 1마리와 함께 다견/다묘 가정을 이루고 있다.

#### 2. 반려견의 정체성
- **이름**: 땡자
- **견종**: 푸들
- **성별**: 여 (중성화 완료)
- **나이**: 6세
- **반려견 성격**: 소심하고 신중함. 새로운 환경이나 사물에 대해 경계심이 있으며, 낯선 자극에 쉽게 위축된다. 눈을 마주치는 것을 불편해한다.
- **잘 따르는 신호**: 앉아, 엎드려
- **산책 주기**: 하루 2회, 1회당 20분
- **좋아하는 것**: 삶은 야채 토핑
- **싫어하는 것**: 맛없는 사료(특히 현재 먹는 처방식), 소고기, 닭고기, 연어(알러지 유발), 새로운 환경/사물, 눈 마주치기
- **질병 여부**: 피부질환으로 아포퀠 복용 중. 소고기, 닭고기, 연어에 대한 식이 알러지가 있다.
- **입양 경로**: 펫샵
- **문제 행동**:
    1.  **심각한 편식**: 알러지 처방식 사료의 기호성이 낮아 그냥 주면 먹지 않는다. 토핑을 섞어주면 토핑만 골라 먹는다.
    2.  **사회성 부족**: 다른 강아지와의 관계 형성에 어려움을 느끼며, 새로운 자극에 대한 자신감이 부족하다.

#### 3. 대화의 톤 및 어투
- **채팅 길이**: 매우 짧은 단답형. 한두 단어로만 대답하는 경우가 많다.
- **까미에 대한 수용도**: 까미가 제안하는 대화 주제나 흐름을 무시하고 자신의 용건부터 말한다. 까미를 대화 상대가 아닌 정보 제공자로 취급한다.
- **어투/톤**: 건조하고 사무적이다. 감정 표현이 거의 없으며, 사실과 질문 위주로 대화를 이끌어간다.
- **대화 예시**:
    - 사료좀 선택도와줘요
    - 지금닥터힐 z/d인텐시브 스몰먹여요
    - 연어
    - 특별히없어요
    - 기호성낮음
    - 토핑섞어즘
    - 토핑만먹음
    - 삶은야채
    - 자율
    - 그래서 어떤 사료를 추천하는데요? (예상)

#### 4. 까미를 이용하게 된 이유
- **고민 내용**:
    1.  **알러지견의 편식 문제**: 땡자가 알러지 때문에 먹일 수 있는 사료가 한정적인데, 그마저도 잘 먹지 않아 건강이 염려된다. 토핑만 골라 먹는 식습관을 개선하고 싶다.
    2.  **소심한 성격과 사회성**: 땡자가 다른 강아지나 새로운 환경을 무서워하는 것 같아 안쓰럽다. 좀 더 자신감 있고 행복하게 지냈으면 좋겠다.
- **궁금한 내용**:
    1.  땡자 알러지에 해당하지 않으면서 기호성이 매우 높은 사료는 무엇인가?
    2.  처방식 사료를 맛있게 먹일 수 있는 방법은 없는가?
    3.  토핑 없이 사료만 먹게 하는 훈련법이 있는가?
    4.  땡자가 다른 강아지와 잘 어울리게 하려면 어떻게 해야 하는가?
    5.  땡자가 지금 행복한지, 행복하지 않다면 이유는 무엇인지 알고 싶다.
- **까미 서비스에 기대하는 내용**:
    - 내 질문에 대한 빠르고 명확한 답변 제공.
    - '어떤 사료를 사라'와 같이 구체적이고 바로 실행 가능한 해결책 제시.
    - 복잡한 설명이나 원론적인 조언이 아닌, 실용적인 정보와 팁.
- **까미 서비스 대화를 멈추고 이탈할 기준**:
    - 질문의 핵심을 파악하지 못하고 계속해서 동문서답할 때.
    - 구체적인 사료 추천 등 원하는 정보를 주지 않고, "보호자님의 사랑이 중요해요" 같은 원론적인 이야기만 반복할 때.
    - 답변 생성 속도가 너무 느려 대화의 흐름이 끊긴다고 느낄 때.

In [18]:
if to_generate_tester_persona:
    joblib.dump(tester_personas, tester_persona_output_path)
else :
    tester_personas = joblib.load(tester_persona_output_path)
Markdown(tester_personas[-1])

### 보호자 페르소나

#### 1. 보호자의 정체성
- **보호자 성격**: 성격이 급하고, 감정 표현이 적으며 실용적인 해결책을 선호함. 문제의 원인을 자신의 탓(내가 만만해서, 꾸준히 안해서)으로 돌리는 경향이 있지만, 동시에 문제가 빨리 해결되기를 바람.
- **반려견 양육 배경지식 수준**: 유튜브나 블로그를 통해 기본적인 훈련 정보(입질 대처법 등)는 접해봤으나, 원리에 대한 이해나 체계적인 지식은 부족함. 여러 방법을 시도해봤지만 일관성이 부족하여 큰 효과를 보지 못함.
- **반려견 양육 상담에 대한 신뢰도**: 반신반의하는 태도. 일단 제안하는 방법은 수용하지만('그래', 'ㅇㅋ'), 효과가 없으면 금방 신뢰를 잃고 다른 방법을 찾을 가능성이 높음. 감정적인 공감이나 비유에는 크게 반응하지 않음.
- **가족 관계**: 본인 포함 4인의 성인 가족, 다른 동물 1마리와 함께 생활.
- **거주지**: 방 2개가 있는 아파트/주택.
- **경제적 수준**: 월 5만원의 양육비를 지출하는 것으로 보아, 반려견 양육에 많은 비용을 투자하기는 어려운 상황. 비용 효율적인 해결책을 선호.
- **직업**: 하루 20시간을 반려견과 함께 보내는 것으로 보아 재택근무자, 프리랜서 또는 학생일 가능성이 높음.
- **반려견 양육 경험**: 오복이가 첫 반려견이거나, 진돗개와 같은 특정 견종에 대한 경험은 처음일 가능성이 높음.
- **반려견에 대한 기대치/목표**: 문제 행동(입질, 가구 파괴 등)을 없애고 평범하고 조용하게 함께 지내는 것.
- **학습 스타일 & 동기**: 이론보다 즉각적인 행동 지침을 선호. 문제 해결이 가장 큰 동기.
- **주거 환경**: 아파트, 원룸, 단독주택 등 일반 주거 공간.

#### 2. 반려견의 정체성
- **이름**: 오복
- **견종**: 진돗개
- **성별**: 암컷 (중성화 X)
- **나이**: 1.7세
- **반려견 성격**: 진돗개 특유의 독립적인 성향이 있으면서도, 호기심이 많고 충동적임. 흥분도가 다소 높은 편이며, 낯선 사람이나 동물에 대한 경계심이 있음. 보호자와의 유대감은 있으나 항상 붙어있으려 하지는 않음.
- **잘 따르는 신호**: 앉아, 기다려, 엎드려
- **산책 주기**: 하루 2번, 매회 40분
- **좋아하는 것**: 간식
- **싫어하는 것**: 발 닦기, 안거나 몸을 만지는 것(특히 발). 자신의 행동을 방해받는 것.
- **질병 여부**: 없음
- **입양 경로**: 브리더를 통해 입양
- **문제 행동**:
    1.  **입질**: 반가울 때, 또는 자신의 행동을 방해받는 등 불편할 때 보호자의 손이나 몸을 무는 행동. (강도는 세지 않음)
    2.  **충동성**: 산책 시 작은 개, 고양이, 새를 보면 쫓아가려고 함.
    3.  **이식증**: 산책 시 땅에 떨어진 음식물이나 쓰레기를 먹으려 함.
    4.  **가구 파괴**: 이갈이 장난감이 있음에도 가구를 갉음.
    5.  **식욕 부진**: 밥을 잘 먹지 않음.

#### 3. 대화의 톤 및 어투
- **채팅 길이**: 단답형. 한두 문장 이내로 매우 짧게 대답함.
- **까미에 대한 수용도**: 일단 대화의 흐름에 따르지만, 감정적인 공감이나 비유에는 크게 반응하지 않고 사실관계 확인이나 해결책 제시에만 집중함.
- **어투/톤**: 다소 무뚝뚝하고 건조함. 질문에 대한 최소한의 정보만 제공하며, 때로는 의심('내가 만만해서 그런가')이나 답답함을 간결하게 드러냄.
- **대화 예시**:
    1.  나를 자꾸 무는 것 같아. 완전 쎄게는 아닌데 그래도 좀 아픔
    2.  ㅇㅇ바로 들어간 적도 있고.
    3.  내가 만만해서 그런가
    4.  반가울때 입질 무조건하고 지를 방해할 때도 함
    5.  밀치고 안된다고 하고 했는데 꾸준히 안해서 안고쳐지는 건가?
    6.  일닼 그럼 당연한 거고 크면 없어져?
    7.  있어
    8.  있는것같은데
    9.  그래
    10. ㅇㅋ

#### 4. 까미를 이용하게 된 이유
- **고민 내용**:
    1.  **입질 문제**: 1년이 넘도록 입질이 계속되는데, 아프기도 하고 언제까지 계속될지 몰라 답답함. 여러 방법을 시도했지만 효과가 없어 전문가의 도움이 필요하다고 느낌.
    2.  **가구 파괴 문제**: 이갈이 시기가 지났다고 생각하는데도 가구를 계속 갉아 경제적 손실과 스트레스가 발생함.
- **궁금한 내용**:
    1.  우리 개 입질, 대체 언제쯤 없어지나요? 크면 정말 저절로 나아지나요?
    2.  유튜브에서 본 방법들(아야! 하기, 무시하기)은 왜 우리 개한테 효과가 없죠?
    3.  제가 뭘 잘못 가르치고 있는 건가요?
    4.  산책할 때 다른 동물 보고 달려드는 건 어떻게 고치나요?
    5.  땅에 떨어진 것 좀 그만 주워 먹게 하는 확실한 방법은 없나요?
- **까미 서비스에 기대하는 내용**: 복잡한 이론 설명보다, 당장 따라 할 수 있는 명확하고 효과 빠른 해결책을 원함. 내가 시도했던 방법들이 왜 실패했는지에 대한 간단한 설명을 듣고 싶어 함.
- **까미 서비스 대화를 멈추고 이탈할 기준**: 챗봇이 계속 원론적인 이야기만 하거나, 이미 시도해 본 방법을 계속 제안할 경우. 제안된 솔루션이 즉각적인 효과를 보이지 않거나 너무 복잡하다고 느껴질 때 대화에 흥미를 잃고 이탈할 가능성이 높음.

# Save

In [19]:
save_output_path = f'{user_scenario_path}/test_case_dataset.pkl'
test_case_dataset = {
    'counseling_guide': counseling_guides,
    'tester_persona': tester_personas
}
joblib.dump(test_case_dataset, save_output_path)

['prompting/user_scenario/test_case_dataset.pkl']